<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;a:link{color: white}">
    <h1 style='color:GhostWhite;'>Part 3: Should This Loan be Approved or Denied ?</h1>
    This is Part 3 of notebook <a style="color:yellow" href="https://www.kaggle.com/code/josephramon/sba-xgboost-model">Part 1: Should This Loan Be Approved or Denied ?</a><br><br>
    This notebook is divided into 3 main parts:<br>
<ul>
<li><a style="color:Gold;" href="#part1"><b>1. tf.Keras Binary Classification Model</b></a></li>
<li><a style="color:Gold;" href="#part2"><b>2. Optuna Hyperparameter Tuning</b></a> </li>
<li><a style="color:Gold;" href="#part3"><b>3. Keras Tuner Hyperparameter Tuning</b></a> </li>
</ul>
<br>
    <i><b>Output from <a style="color:Gold;" href = "https://www.kaggle.com/code/josephramon/sba-xgboost-model">Part 1 notebook</a> are Input to this notebook.</b></i><br><br>
    The techniques covered here are :<br>
    <p style="color:Gold;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>Deep Learning: keras Binary Classification</b></p>
    <p style="color:Gold;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>keras.experimental.WideDeepModel</b></p>
    <p style="color:Gold;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>Optuna Hyperparameter Tuning for Keras</b></p>
    <p style="color:Gold;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>Keras Tuner Hyperparameter Tuning</b></p>
    </div>

<div class="alert alert-block alert-success">  
    <b>Dataset Source</b><br>
    <a href="https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied">U.S. Small Business Administration (SBA) Dataset</a> - all information about the dataset can be found in this link 
</div>

<div class="alert alert-block alert-warning" style="color:DarkSlateBlue">
    Kaggle's <b>Tensor Processing Units (TPUs)</b> are accelerators ideal for large workloads.  Using a TPU is a bit like using eight GPUs at once.<br><br>
    <b>Model Training Time Comparison in Kaggle :<br>
        200 epochs with early stopping, and our X_train dataset shape is almost a million rows after oversampling.</b><br>
    <li>With TPU - about 10 minutes</li>
    <li>With GPU - about 1 hour</li>
    <li>Without TPU or GPU - more than 24 hours</li>
    <br>
For running Optuna or Keras Tuner here, it would be ideal to use TPU. One can still just use GPU, but it will take about 18 to 20 hours for an Optuna 30-trial run, for example.<br><br>
Optuna is faster than Keras Tuner Bayesian Optimization.
</div>

<a id="toc"></a>
<h2>Table Of Contents</h2>
<ul>
    <li><a style="color:DarkSlateGrey" href="#paths_and_flags">Paths and Flags</a></li>
    <li><a style="color:DarkSlateGrey" href="#libraries">Libraries</a></li>   
    <li><a style="color:DarkSlateGrey" href="#functions">Custom Functions And Classes</a></li>
    <li><a style="color:DarkSlateGrey" href="#prepdataset_class">PrepDataset Class</a></li>
    <li><a style="color:DarkSlateGrey" href="#clr_class">Cyclical Learning Rate Class</a></li>
    <li><a style="color:DarkSlateGrey" href="#keras_binary_class">Keras Binary Classification Class</a></li>
    <li><a style="color:DarkSlateGrey" href="#tp_class">TestPredict Class</a></li>
    <li><a style="color:DarkSlateGrey" href="#cr_class">CompareResults Class</a></li>
    <li><a style="color:DarkSlateGrey" href="#optuna_class">Optuna Class</a></li>
    <li><a style="color:DarkSlateGrey" href="#keras_tuner_class">Keras Tuner Class</a></li>
    <br>
    <li><a style="color:DarkSlateGrey" href="#part1">Part 1. tf.Keras Models</a></li>
    <ul>
        <li><a style="color:DarkSlateGrey" href="#binary_classification">Binary Classification tf.keras model</a></li>
        <ul>
            <li><a style="color:DarkSlateGrey" href="#test_unseen">Test Binary Classification Model with Unseen Dataset</a></li>
            <li><a style="color:DarkSlateGrey" href="#uit">Test Binary Classification Model with Single User Inputs</a></li>
        </ul>
        <li><a style="color:DarkSlateGrey" href="#binary_classification_wide_and_deep">Binary Classification Wide and Deep tf.keras model</a></li>
        <ul>
            <li><a style="color:DarkSlateGrey" href="#test_unseen_wd">Test keras.experimental.WideDeepModel with Unseen Dataset</a></li>
            <li><a style="color:DarkSlateGrey" href="#uit_wd">Test keras.experimental.WideDeepModel with Single User Inputs</a></li>
        </ul>
        <li><a style="color:DarkSlateGrey" href="#compare_bc_wd">Score Comparison : Binary Classification vs WideAndDeep Models</a></li>
    </ul>   
    <br>
    <li><a style="color:DarkSlateGrey" href="#part2">Part 2. Optuna Hyperparameter Tuning</a></li>
    <ul>
        <li><a style="color:DarkSlateGrey" href="#optuna_study">Optuna Study</a></li> 
        <li><a style="color:DarkSlateGrey" href="#optuna_metrics">Optuna Study Metrics</a></li>
        <li><a style="color:DarkSlateGrey" href="#try_best_hp">Test Optuna Best Trial Hyperparameters</a></li>
        <li><a style="color:DarkSlateGrey" href="#optuna_comparison">Optuna Tuning Comparison</a></li>
    </ul>
    <br>
    <li><a style="color:DarkSlateGrey" href="#part3">Part 3. Keras Tuner Hyperparameter Tuning</a></li>
    <ul>
        <li><a style="color:DarkSlateGrey" href="#kt_study">Keras Tuner Search</a></li> 
        <li><a style="color:DarkSlateGrey" href="#kt_metrics">Keras Tuner Metrics</a></li>
        <li><a style="color:DarkSlateGrey" href="#try_best_hp">Test Keras Tuner's Best Trial Hyperparameters</a></li>
        <li><a style="color:DarkSlateGrey" href="#kt_comparison">Keras Tuner Tuning Comparison</a></li>
    </ul>
</ul>

<a id="paths_and_flags"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Paths and Flags</b></div>

In [2]:
import os
'''
kaggle_flag :
   0 - if running outside Kaggle (e.g. Jupyter Notebook), change inputdir & workdir to your 
       own path
   1 - if running as a Kaggle notebook
'''
# Change this logic to your own if needed
if os.path.exists('../usr/lib/myfuncs/myfuncs.py'):
    kaggle_flag = 1
    print('Running a Kaggle notebook')
else:
    kaggle_flag = 0
    print('Not running a Kaggle notebook')

# alert_flag - change to 0 for no sound alert, 1 for sound alert after long running cells
alert_flag = 0

'''
Change flag below as needed:
   1 to run Optuna
   0 Do Not run Optuna
'''
optuna_flag = 1

'''
Change flag below as needed:
   1 to run Keras Tuner
   0 Do Not run Keras Tuner
'''
kt_flag = 0

# GPU is automatically detected if activated

#---------------------------------------------------------------------------------------#

if kaggle_flag == 1:             # Kaggle
    # Part 1 Notebook's workdir contents are input for this notebook
    inputdir  = "../input/sba-xgboost-model/"        
    workdir  = "./"
    final_ds  = f'{inputdir}sba_final.csv.feather'  # imported from Part 1 Notebook
    final_csv = f'{inputdir}sba_final.csv'          # imported from Part 1 Notebook
    functions_path = "../usr/lib/myfuncs/myfuncs.py"
else:
    inputdir  = "C:\\Python\\Python_Data_Science_Exercises\\datasets\\"
    workdir  = "C:\\Python\\Python_Data_Science_Exercises\\datasets\\"
    final_ds  = f'{inputdir}sba_final.csv.feather'
    final_csv = f'{inputdir}sba_final.csv'
    functions_path = 'C:\\Python\\Python_Data_Science_Exercises\\mylibs\\'

audio_path="https://www.soundjay.com/misc/sounds/tablet-bottle-1.mp3" # for alert

print(f'kaggle_flag : {kaggle_flag}')
print(f'alert_flag  : {alert_flag}')
print(f'optuna_flag : {optuna_flag}')
print(f'kt_flag     : {kt_flag}')

Not running a Kaggle notebook
kaggle_flag : 0
alert_flag  : 0
optuna_flag : 1
kt_flag     : 0


<a id="libraries"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Libraries</b></div>

In [3]:
from IPython.display import clear_output   # to be able to use clear_output(wait=True)
def install_packages():
    print('Please wait, package installations started, if needed')
    libs = ['scikit-learn', 'seaborn', 'numpy','matplotlib', 'tensorflow','torch','joblib',
            'psutil','imbalanced-learn','pyarrow','pyttsx3','sweetviz', 'optuna', 'pydotplus',
            'keras-tuner']
    
    piplist = !pip list
    for i in range(len(libs)):
        if not piplist.grep(libs[i]):
            !pip3 install {libs[i]}

    clear_output(wait=True)
    print('Package installations completed')

install_packages()

Package installations completed


In [4]:
print('Please wait, importing packages')
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

from IPython.display import Audio, display
from IPython.core.display import HTML

import gc
import shutil
import psutil
import os
import sys
import pickle
import joblib
import copy                     # for deepcopy()
import datetime as dt
from time import sleep

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pyttsx3
#import hashlib

import torch                    # for clearing GPU cache
%matplotlib inline

import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.constraints import unit_norm

import pydot
import pydotplus
from tensorflow.keras.utils import plot_model

import keras_tuner as kt
from mock import patch

# for preparing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler   #, OneHotEncoder
#from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer

# for CyclicLR class
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

clear_output(wait=True)
print('Package imports completed')

Package imports completed


In [8]:
# Kernel must be restarted if Keras Tuner is upgraded
# importlib.reload and %autoreload do not work, so manually restart 
# This check is basically for Kaggle which has an older version of Keras Tuner, as at May 2022
if kt.__version__ < '1.0.3':
    !pip3 install --upgrade keras-tuner
    clear_output(wait=True)
    print('keras-tuner Package upgrade completed.  KERNEL RESTART NEEDED FOR NOTEBOOK.')
else:
    print(f'keras-tuner version {kt.__version__} is fine')

keras-tuner version 1.1.2 is fine


In [10]:
# keras-tuner version should be 1.0.3 and up
assert kt.__version__ >= '1.03',\
    "kers-tuner version must be 1.0.3+. RESTART KERNEL if already upgraded."

print(f'keras-tuner __Version__ : {kt.__version__}')
print()
!pip3 show keras-tuner

keras-tuner __Version__ : 1.1.2

Name: keras-tuner
Version: 1.1.2
Summary: Hypertuner for Keras
Home-page: https://github.com/keras-team/keras-tuner
Author: The KerasTuner authors
Author-email: kerastuner@google.com
License: Apache License 2.0
Location: c:\programdata\anaconda3\lib\site-packages
Requires: numpy, tensorboard, kt-legacy, ipython, packaging, requests
Required-by: 


In [ ]:
# enable garbage collector if disabled
(gc.isenabled() == False) and gc.enable();

<a id="functions"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Custom Functions and Classes</b></div>

In [ ]:
sys.path

In [ ]:
# import custom functions
# RESTART kernel if myfuncs is modified
if functions_path not in sys.path:
    sys.path.append(functions_path)
from myfuncs import *

print('Custom functions import completed')

<div class="alert alert-block alert-info">
<b>Custom functions and classes in <a style="color:ForestGreen" href="https://www.kaggle.com/code/josephramon/myfuncs" target="_blank">myfuncs.py</a></b>.<br>  
In Kaggle, myfuncs.py is set up as a <b>Utility Script</b> in /usr/lib<br>
<ul>
    <li>is_kaggle_gpu_enabled()</li>
<li>clear_gpu(tree_method='gpu_hist')</li>
<li>reduce_mem_usage(df, print_info = True, use_float16=False)</li>
<li>runtime(rt1,rt2)</li>
<li>create_download_link(title = "Download ", filename = "data.csv")</li>
<li>GetRam()</li>
<li>convertFloatToDecimal(f=0.0, precision=2)</li>
<li>formatFileSize(size, sizeIn, sizeOut, precision=0)</li>
<li>check_cols_with_nulls(df)</li>
<li>check_infinity_nan(df, dfname)</li>
<li>fixvals(val)</li>
<li>model_eval(y_valid,predictions, cmDisplay='False')</li>
<li>plot_features(booster, figsize)</li>
<li>make_mi_scores(X, y)</li>
<li>plot_mi_scores(scores)</li>
<li>GetSweetVizReport(df, workdir, kaggle_flag)</li>
<li>SetVoice(kaggle_flag)</li>
<li>InitTPUStrategy()</li>
<li>ZipDir(zippath)</li>
<li>class color
</div>

In [ ]:
gpu_enabled = is_kaggle_gpu_enabled()

sleep(5)
clear_output(wait=True)
print("Is GPU enabled ? ", gpu_enabled)

In [ ]:
''' 
Set up voice object.  Used in different areas of notebook to indicate completion of long processes.
'''
engine = SetVoice(kaggle_flag)
print('Voice engine set up')

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="prepdataset_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>PrepDataset Class</b></div>

In [ ]:
class PrepDataset:  
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.X_train, self.y_train = None, None
        self.X_valid, self.X_test = None, None
        self.y_valid, self.y_test = None, None
    
    # oversampling method
    def osample(self, print_info = True):
        # define oversampling strategy
        oversample = RandomOverSampler(sampling_strategy='minority') 
        if print_info == True:
            print('X size : ', len(self.X))
            print('y size : ', len(self.y))
        # fit and apply the transform
        X_over, y_over = oversample.fit_resample(self.X, self.y)

        # summarize class distribution
        if print_info == True:
            print(f'y Before Oversampling -> 1 : {Counter(self.y)[1]}, 0 : {Counter(self.y)[0]}')
            print(f'y After Oversampling  -> 1 : {Counter(y_over)[1]}, 0 : {Counter(y_over)[0]}')
        
        # update X and y with the oversampled results 
        self.X = X_over
        self.y = y_over
    
    def split_data(self, X_size = 0.7):   
        # Split Data into Train:Validate:Test
        
        # train_size=X_size
        # In the first step, we will split the data in training and remaining dataset
        self.X_train, X_rem, self.y_train, y_rem = train_test_split(self.X, self.y,
                                        train_size = X_size, random_state=48) 

        # Now since we want the valid and test size to be equal,
        # we have to define valid_size=0.5 (that is 50% of remaining data)
        # test_size = 0.5

        self.X_valid, self.X_test, self.y_valid, self.y_test = train_test_split(X_rem,y_rem,
                                        test_size=0.5, random_state=48)
        
        return {'X_train':self.X_train, 'y_train':self.y_train,
                'X_valid':self.X_valid, 'y_valid':self.y_valid,
                'X_test':self.X_test, 'y_test':self.y_test}

    def Scale_Data(self):
        # Scale dataset
        #scalar = MinMaxScaler()
        scalar = StandardScaler()
        scalar.fit(self.X_train)
        self.X_train = scalar.transform(self.X_train)
        self.X_valid = scalar.transform(self.X_valid)
        self.X_test = scalar.transform(self.X_test)

        # transform into numpy arrays
        self.y_train, self.y_valid, self.y_test =\
            self.y_train.values, self.y_valid.values, self.y_test.values
        
    def Scale_Data_Pipeline(self):
        # Scale dataset using a pipeline, just another approach (the other one is Scale_Data())
        features_num = list(self.X)
        #print(features_num)

        # features_cat     # we don't have any category types

        transformer_num = make_pipeline(
            #SimpleImputer(strategy="constant"), # there are a few missing values
            StandardScaler(),
            #MinMaxScaler(),
            )
        '''
        transformer_cat = make_pipeline(
            SimpleImputer(strategy="constant", fill_value="NA"),
            OneHotEncoder(handle_unknown='ignore'),
        )
        '''
        preprocessor = make_column_transformer(
            (transformer_num, features_num),
            #(transformer_cat, features_cat),
        )
    
        # Scale Dataset
        self.X_train = preprocessor.fit_transform(self.X_train)
        self.X_valid = preprocessor.transform(self.X_valid)
        self.X_test  = preprocessor.transform(self.X_test)

        # convert to numpy arrays
        self.y_train, self.y_valid, self.y_test =\
            self.y_train.values, self.y_valid.values, self.y_test.values

    def Scale_Data_Yeo_Johnson(self):
        features_num = list(self.X)

        transformer_num = make_pipeline(
            PowerTransformer(method = 'yeo-johnson'),
            )
        
        preprocessor = make_column_transformer(
            (transformer_num, features_num),
        )
        
        # Scale Dataset
        self.X_train = preprocessor.fit_transform(self.X_train)
        self.X_valid = preprocessor.transform(self.X_valid)
        self.X_test  = preprocessor.transform(self.X_test)

        # convert to numpy arrays
        self.y_train, self.y_valid, self.y_test =\
            self.y_train.values, self.y_valid.values, self.y_test.values
        
    def Scale_Data_Np_Log(self):
        # only features that are not 0/1
        features_num = list(self.X)
        
        # Scale Dataset
        self.X_train = np.log(self.X_train+1)
        self.X_valid = np.log(self.X_valid+1)
        self.X_test  = np.log(self.X_test+1)

        # convert to numpy arrays
        self.y_train, self.y_valid, self.y_test =\
            self.y_train.values, self.y_valid.values, self.y_test.values
        
print('PrepDataset class defined')

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="clr_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Cyclical Learning Rate Class</b><br>
    <i>source: </i><a style="color:MidnightBlue;" href = "https://github.com/bckenstler/CLR/blob/master/clr_callback.py">https://github.com/bckenstler/CLR/blob/master/clr_callback.py<a>
</div>

In [ ]:
class CyclicLR(callbacks.Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[callbacks.clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[callbacks.clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        #print line added by me
        #print(f'\n{color.bdgreen}Learning Rate: {K.eval(self.model.optimizer.lr)}{color.end}\n')
        if self.scale_mode == 'cycle':
            return self.base_lr +\
                (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr +\
                (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr()) 
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
        
print('CyclicLR class defined')

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

In [ ]:
tf.keras.metrics.RootMeanSquaredError(
    name='root_mean_squared_error', dtype=None
)

<a id="keras_binary_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Keras Binary Classification Class</b>
</div>

In [ ]:
class KerasBinaryClassification(PrepDataset):
    def __init__(self, X, y):
        # use class PrepDataset's __init__ variables
        PrepDataset.__init__(self, X, y)
        
        #-----------------------------
        # Initialize default values
        #-----------------------------
        
        # Model Configuration
        self.UNITS = 2 ** 11 # 2048
        self.ACTIVATION = 'swish'
        self.DROPOUT = 0.1
        #self.kernel_constraint_val = 3   # for dropout

        self.EPOCHS = 200
        self.EARLY_STOPPING_ROUNDS = self.EPOCHS * 0.1   # 10%
    
        # Training Configuration
        # To go fast on a TPU, increase the batch size. The rule of thumb is to use batches of 128 
        # elements per core (ex: batch size of 128*8=1024 for a TPU with 8 cores).
        self.BATCH_SIZE_PER_REPLICA = 2 ** 11 # powers of 128 are best
        
        # For model.compile
        # To reduce Python overhead and maximize the performance of your TPU, pass in the 
        # argument steps_per_execution to model.compile. In this example, it increases throughput
        self.steps_per_execution = 256
        self.learning_rate = None
        self.compile_metrics = ['AUC', 'binary_accuracy', 'Precision', 'Recall']
        
        self.kt_run = False          # becomes True in WideAndDeepModel(), if Keras Tuner was run
        
        # For model.fit
        self.clr_triangular = None    # for CyclicLR object instantiated in PrepKSModel() below
        self.model = None
        #batch_size = BATCH_SIZE_PER_REPLICA * tpu_strategy.num_replicas_in_sync
        #steps_per_epoch = len(self.X_train) // batch_size
        #validation_steps = len(self.X_valid) // batch_size
        
        self.fit_callbacks = None
        
        # early_stopping
        self.early_stopping, self.lr_schedule = None, None
        
        self.input_shape = None
        
        self.n_layers = 6
        
        # wide and deep models
        self.wide, self.deep, self.wide_and_deep = None, None, None
    
    def dense_block(self, units, activation, dropout_rate, l1=None, l2=None):
        def make(inputs):
            x = layers.Dense(units)(inputs)
            x = layers.Activation(activation)(x)
            x = layers.BatchNormalization()(x)                # ideally after Activation
            x = layers.Dropout(dropout_rate)(x)               
            return x
        return make

    # Create a model using the Sequential method
    def create_seq_model(self):
        '''instantiating the model in the strategy scope creates the model on the TPU'''
        with tpu_strategy.scope():
            model = keras.Sequential()
            model.add(BatchNormalization(input_shape=self.input_shape))
            for i in range(self.n_layers):
                model.add(Dense(self.UNITS, activation=self.ACTIVATION))
                                #kernel_constraint=unit_norm()))
                                #kernel_constraint=max_norm(self.kernel_constraint_val)))
                model.add(BatchNormalization())
                model.add(Dropout(self.DROPOUT))
            model.add(Dense(1, activation='sigmoid'))
            
            self.model = model
        
        # Prepare Keras model (compile, callbacks for early_stopping and lr_schedule)
        self.PrepKSModel()
    
        return self.model
    
    # create a model using the Functional API method, which is more flexible than Sequential()
    # e.g. can have more inputs and outputs
    def create_func_api_model(self):
        '''instantiating the model in the strategy scope creates the model on the TPU'''
        with tpu_strategy.scope():
            inputs = keras.Input(shape = self.input_shape)
        
            for i in range(self.n_layers):
                if i == 0:
                    x = self.dense_block(self.UNITS, self.ACTIVATION, self.DROPOUT)(inputs)
                else:
                    x = self.dense_block(self.UNITS, self.ACTIVATION, self.DROPOUT)(x)

            outputs = layers.Dense(1, activation='sigmoid')(x)
            self.model = keras.Model(inputs=inputs, outputs=outputs)
        
        # Prepare Keras model (compile, callbacks for early_stopping and lr_schedule)
        self.PrepKSModel()

        return self.model
 
    # create a wide_and_deep model using keras.experimental.WideDeepModel and Functional API
    def create_wide_and_deep_model(self):
        '''instantiating the model in the strategy scope creates the model on the TPU'''
        with tpu_strategy.scope():
            # Wide Network
            self.wide = keras.experimental.LinearModel()

            # Deep Network
            inputs = keras.Input(shape = self.input_shape)
            for i in range(self.n_layers):
                if i == 0:
                    x = self.dense_block(self.UNITS, self.ACTIVATION, self.DROPOUT)(inputs)
                else:
                    x = self.dense_block(self.UNITS, self.ACTIVATION, self.DROPOUT)(x)  
            outputs = layers.Dense(1)(x)
            self.deep = keras.Model(inputs = inputs, outputs = outputs)
    
            # Wide and Deep Network
            self.wide_and_deep = keras.experimental.WideDeepModel(
                                        linear_model = self.wide,
                                        dnn_model = self.deep,
                                        activation = 'sigmoid',
                                        )
            self.model = self.wide_and_deep
        # Prepare Keras model (compile, callbacks for early_stopping and lr_schedule)
        self.PrepKSModel()        

        return self.model

    # Prepare Keras model
    def PrepKSModel(self):
        '''Starting with Tensorflow 2.4, model.compile() accepts a new steps_per_execution 
        parameter. This parameter instructs Keras to send multiple batches to the TPU at once. 
        In addition to lowering communications overheads, this gives the XLA compiler the 
        opportunity to optimize TPU hardware utilization across multiple batches. 
    
        For classification, what we want instead is a distance between probabilities, and this is 
        what cross-entropy provides. Cross-entropy is a sort of measure for the distance from one 
        probability distribution to another.  For two-class problems, be sure to use 'binary' 
        versions.
    
        The cross-entropy and accuracy functions both require probabilities as inputs, meaning, 
        numbers from 0 to 1. To covert the real-valued outputs produced by a dense layer into 
        probabilities, we attach a new kind of activation function, the sigmoid activation.
        

        To get the final class prediction, we define a threshold probability. Typically this 
        will be 0.5, so that rounding will give us the correct class: below 0.5 means the class 
        with label 0, and 0.5 or above means the class with label 1. A 0.5 threshold is what Keras 
        uses by default with its accuracy metric.
        '''
        
        tf.keras.metrics.BinaryAccuracy(name='binary_accuracy', dtype=None, threshold=0.5)
        
        if self.learning_rate == None:
            opt = tf.keras.optimizers.Adam()
        else:
            opt = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)

        self.model.compile(
            optimizer = opt,
            loss = 'binary_crossentropy',
            metrics = self.compile_metrics,
            steps_per_execution = self.steps_per_execution,
        )

        # Callbacks when fitting
        '''
        "The learning rate is perhaps the most important hyperparameter. If you have 
        time to tune only one hyperparameter, tune the learning rate."
        '''
        self.clr_triangular = CyclicLR(mode='triangular')
        
        '''
        During training, we'll use the EarlyStopping callback as usual while fitting.
        '''
        self.early_stopping = callbacks.EarlyStopping(
            monitor = 'val_loss',
            mode = 'auto',
            patience = self.EARLY_STOPPING_ROUNDS,
            min_delta = 0.001,
            restore_best_weights = True,
            verbose = 1
        )
        
        # This will not be used at the moment, but instead will use the CyclicLR class
        '''
        We've also defined a learning rate schedule. It's been found that gradually decreasing 
        the learning rate over the course of training can improve performance (the weights 
        "settle in" to a minimum). This schedule will multiply the learning rate by 0.2 if the 
        validation loss didn't decrease after an epoch.
        '''
        self.lr_schedule = callbacks.ReduceLROnPlateau(
            monitor = 'val_loss',
            mode = 'auto',
            patience = 0,
            factor = 0.2,
            min_lr = 0.001,
            verbose = 1
        )
        
        if self.kt_run == False:       # if Keras Tuner is not being run
            self.fit_callbacks = [  
                                    self.early_stopping,
                                    self.clr_triangular
                                 ]
                                    #self.lr_schedule]
        else:
            self.fit_callbacks=[self.early_stopping]

        return self.model
    
    def FitModel(self):
        history = self.model.fit(
                                 self.X_train,
                                 self.y_train,
                                 validation_data = (self.X_valid, self.y_valid),
                                 batch_size = self.BATCH_SIZE_PER_REPLICA,
                                 epochs = self.EPOCHS,
                                 #steps_per_epoch=steps_per_epoch,
                                 #validation_steps=validation_steps,
                                 callbacks = self.fit_callbacks
                                )

        return history
    
    def SavePlotHistory(self, hdf, savename):
        hdf['f1_score'] = 2*((hdf['precision']*hdf['recall'])/(hdf['precision']+hdf['recall']))
        hdf['val_f1_score'] = 2*((hdf['val_precision']*hdf['val_recall'])/
                                 (hdf['val_precision']+hdf['val_recall']))
    
        print()
        print(f'{color.bdgreen}{color.underline}SCORES:{color.end}')
        display(hdf.tail(5)[['loss','auc','binary_accuracy','precision','recall','f1_score']])
        display(hdf.tail(5)[['val_loss','val_auc','val_binary_accuracy',
                     'val_precision','val_recall', 'val_f1_score']])
    
        # Save history
        hdf_save = hdf.copy().reset_index(drop=True)
        hdf_save.to_feather(f'{workdir}{savename}')
        print()
        print(f'{color.bdgreen}History df saved to {workdir}{savename}{color.end}')
        del hdf_save
        gc.collect()
    
        print()
    
        print("{}Minimum Validation Loss: {}{:0.4f}{}".
            format(color.bdblue, color.bdgreen, hdf["val_loss"].min(), color.end))
        print("{}Minimum Loss: {}{:0.4f}{}".
            format(color.bdblue, color.bdgreen, hdf["loss"].min(), color.end))
        print("{}Maximum Validation Accuracy: {}{:0.4f}{}".
            format(color.bdblue, color.bdgreen, hdf['val_binary_accuracy'].max()*100, color.end))
        print("{}Maximum Accuracy: {}{:0.4f}{}".
            format(color.bdblue, color.bdgreen, hdf['binary_accuracy'].max()*100, color.end))
        print("{}Maximum Validation F1-Score: {}{:0.4f}{}".
            format(color.bdblue, color.bdgreen, hdf['val_f1_score'].max()*100, color.end))
        print("{}Maximum F1-Score: {}{:0.4f}{}".
            format(color.bdblue, color.bdgreen, hdf['f1_score'].max()*100, color.end))
        print()
    
        hdf.loc[:, ['loss', 'val_loss']].plot(title='Cross-entropy Loss')
        hdf.loc[:, ['auc', 'val_auc']].plot(title='AUC');
        hdf.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title='Accuracy');
        hdf.loc[:, ['f1_score', 'val_f1_score']].plot(title='F1-Score');
    
        return hdf

print('KerasBinaryClassification class defined')

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="tp_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>TestPredict Class</b>
</div>

In [ ]:
class TestPredict:
    def __init__(self, model):
        self.model = model

    def PreProcessUserInput(self):
        user_input_list = list(self.user_input.values())

        # convert to np array
        features = np.array([user_input_list])

        # since it's only one entry, reshape data so scaling won't return all zeroes
        reshapedData = features.reshape(-1,1)

        scalar = PowerTransformer(method = 'yeo-johnson')
        scalar.fit(reshapedData)
        features = scalar.transform(reshapedData)
        
        # restore to original shape
        features = features.reshape(1,-1)
    
        return features
    
    # called when predicting score for user input data
    def UserInputTestPredict(self,
                             testnum = 0,
                             user_input = None):
        
        # user_input is a dictionary; deepcopy() so original is not affected by any changes
        self.user_input = copy.deepcopy(user_input)
        
        print()
        print(f"{color.bold}User Entry: {testnum}{color.end}")
        display(self.user_input)
        
        features = self.PreProcessUserInput()   # preprocess user input
    
        pred = (self.model.predict(features) >= 0.5).astype("int32")
        
        print(f'{color.bdgreen}Prediction: {pred[0][0]}{color.end}')

        if pred[0][0] == 1:
            print(f'{color.bdblue}Prediction: Approve The Loan{color.end}')
        else:
            print(f'{color.bdred}Prediction: Do Not Approve The Loan{color.end}')
        print()
        
        # reset self.user_input to default for next User Entry test, if any
        self.user_input = copy.deepcopy(user_input)
    
    # called when predicting score for unseen data (X_test)
    def PredictTestData(self,
                        X_test = None,
                        y_test = None,
                        label = "Keras Model"):
        # Get predictions
        print(f'{color.bold}Please wait, preparing predictions{color.end}')

        predictions = (self.model.predict(X_test) >= 0.5).astype("int32")

        #clear_output(wait=True)
        print(f'{color.bdblue}{label} Score:{color.end}')

        asm = (accuracy_score(y_test, predictions.round()) * 100)
        print(f'{color.bdgreen}Accuracy for model: %.2f %%{color.end}' % asm)
        print()    # newline in case the method is run again
    
print('TestPredict Class defined')

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="cr_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>CompareResults Class</b>
</div>

In [ ]:
class CompareResults:
    def __init__(self):
        pass
        
    def BuildCompare(self,
                     hdf1, hdf2,
                     col1name, col2name, titlename
                    ):
    
        data = {col1name:[
                            hdf1["val_loss"].min(),
                            hdf1["val_binary_accuracy"].max()*100,
                            hdf1["val_f1_score"].max()*100,
                            hdf1["loss"].min(),
                            hdf1["binary_accuracy"].max()*100,
                            hdf1["f1_score"].max()*100
                         ],
                col2name:[
                            hdf2["val_loss"].min(),
                            hdf2["val_binary_accuracy"].max()*100,
                            hdf2["val_f1_score"].max()*100,
                            hdf2["loss"].min(),
                            hdf2["binary_accuracy"].max()*100,
                            hdf2["f1_score"].max()*100
                         ]
                }
    
        # Create pandas DataFrame
        df = pd.DataFrame(data, index = [
                                    'Validation Loss',
                                    'Validation Binary Accuracy',
                                    'Validation F1-Score',
                                    'Loss',
                                    'Binary Accuracy',
                                    'F1-Score'
                                        ]
                         )
        print(f'{color.bdblue}{titlename}{color.end}')
        display(df)

        acc_diff = round((hdf2["binary_accuracy"].max()*100) -\
                                   (hdf1["binary_accuracy"].max()*100),2)
        print(f'{color.bdgreen}Binary Accuracy Improvement:{color.end}')
        print(f'{color.bold}Improved by {color.bdblue}{acc_diff} percentage points{color.end}')

print('CompareResults Class defined')

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="optuna_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Optuna Class</b><br>
</div>

In [ ]:
class optuna_tuning(KerasBinaryClassification):  
    def __init__(self, X, y):
        # KerasBinaryClassification will in turn use class PrepDataset's __init__ variables
        KerasBinaryClassification.__init__(self, X, y)

        self.input_shape = None
        self.nn_trials = None
        self.nn_timeout = None        # 60 seconds * 120 = 2 hours
        self.gt = None                # just for calculating elapsed optuna training time
        self.hdg1 = None
        self.old_time = dt.datetime.now()
        self.current_time = self.old_time.strftime("%H:%M:%S")
        self.end_time = None
    
    # for printing only the best values
    def logging_callback(self, study, frozen_trial):
        previous_best_value = study.user_attrs.get("previous_best_value", None)
        if previous_best_value != study.best_value:
            study.set_user_attr("previous_best_value", study.best_value)
            clear_output(wait=True)
            print(self.hdg1)
            print()        
            self.text_out=\
                "{}Trial {} done with best value: {}{}{} and parameters: {}{}{}. ".format(
                    color.bdblue,
                    frozen_trial.number,
                    color.bdgreen,
                    frozen_trial.value,
                    color.bdblue,
                    color.green,
                    frozen_trial.params,
                    color.end
                    )
            print(self.text_out)
            
            # Writing to file
            with open(f"{workdir}ks_optuna_study_log.txt", "a") as os_log:
                os_log.write('\n\n')
                os_log.write(f"Current Ram Used: {GetRam()} %\n")
                os_log.write(self.text_out)
    
    # save study
    def save_study(self, study, frozen_trial):
        joblib.dump(study, f"{workdir}ks_optuna_study_callbacks.pkl")   # save study

    # define the model
    def create_model(self, trial):
        self.n_layers = trial.suggest_categorical("n_layers", [6, 8, 10])
        self.UNITS = trial.suggest_categorical("units",
                                            [512, 1024, 2048])
        self.ACTIVATION = trial.suggest_categorical("activation",
                                            ["relu", "swish"])
        self.DROPOUT = trial.suggest_categorical("dropout_rate",
                                            [0.1, 0.3, 0.5])
       
        self.create_func_api_model()   # create model with Functional API method
            
        # We wont' work on the learning_rate with Optuna, it will be handled by the lr scheduler

        # Prepare Keras model (compile, callbacks for early_stopping and lr_schedule)
        #self.PrepKSModel()
        
        print()
        param_info =\
            "{}Current Trial: {}Layers: {} | Units: {} | ACTIVATION: {} | DROPOUT: {}{}".format(
                color.bold,
                color.bdblue,
                self.n_layers,
                self.UNITS,
                self.ACTIVATION,
                self.DROPOUT,
                color.end
                )
        print(param_info)

    # Optuna objective function 
    def objective_keras(self, trial):
        # Clear clutter from previous session graphs.
        keras.backend.clear_session()
        
        # clear outputs, reprint saved data
        if trial.number > 0:
            clear_output(wait=True)
            gc.collect()
            print(f"\n{color.bold}Time Started = {color.bdgreen}{self.current_time}{color.end}")
            if self.end_time != None:
               print(f'{color.bold}Optuna Trials will end around: {color.bdred}{self.end_time}\
                   {color.end}\n') 
            print(self.hdg1)
            print()
            print(self.text_out)
        else:
            print(f"\n{color.bold}Time Started = {color.bdgreen}{self.current_time}{color.end}")
            if self.nn_timeout == None:
                self.hdg1 =\
                    "{}Number of Trials: {} | Epochs: {} | Early Stopping Rounds: {}{}".format(
                        color.bdgreen,
                        self.nn_trials,
                        self.EPOCHS,
                        self.EARLY_STOPPING_ROUNDS,
                        color.end
                        )
            else:
                self.hdg1 =\
                    "{}TimeOut: {} | Epochs: {} | Early Stopping Rounds: {}{}".format(
                        color.bdgreen,
                        self.nn_timeout,
                        self.EPOCHS,
                        self.EARLY_STOPPING_ROUNDS,
                        color.end
                        )
                self.end_time = self.old_time + dt.timedelta(seconds = self.nn_timeout)
                print(f'{color.bold}Optuna Trials will end around: {color.bdred}{self.end_time}\
                    {color.end}\n')
            print(self.hdg1)
            
        # Generate our trial model
        self.create_model(trial)
    
        print()
        print(f"Current Ram Used: {GetRam()} %")
        rt2=dt.datetime.now()
        print(f'{color.bdgreen}Total Elapsed Time from Training Start: {color.end}', end='')
        runtime(self.gt, rt2)  
        print()
        print(f'Running Trial {trial.number}')
        print()
        
        # Fit the model on the training data.
        # The KerasPruningCallback checks for pruning condition every epoch.
        #************************************************************************************  
        
        history = self.FitModel()
        
        #************************************************************************************

        predictions = (self.model.predict(self.X_valid) >= 0.5).astype("int32")

        asm = (accuracy_score(self.y_valid, predictions.round()) * 100)
        trial.report(asm, 1)
            
        #rmse = metrics.mean_squared_error(self.y_valid, predictions,squared=False)
        #trial.report(rmse, 1)
            
        if trial.should_prune():
            text_prune = f'{color.bold}Trial {trial.number} pruned{color.end}'
            # Writing to file
            with open(f"{workdir}ks_optuna_study_log.txt", "a") as os_log:
                os_log.write('\n')
                os_log.write(text_prune)
                
            raise optuna.TrialPruned()

        text_dtl = "Trial {} finished with parameters: {}. ".format(
            trial.number,
            trial.params
            )
        # Writing to file
        with open(f"{workdir}ks_optuna_study_log.txt", "a") as os_log:
            os_log.write('\n\n')
            os_log.write(f"Current Ram Used: {GetRam()} %\n")
            os_log.write(text_dtl)
            
        rt2=dt.datetime.now()     
         
        return asm

print('optuna_tuning class defined')

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="keras_tuner_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Keras Tuner Class</b><br>
</div>

In [ ]:
class keras_tuning(KerasBinaryClassification):  
    def __init__(self, X, y):
        # KerasBinaryClassification will in turn use class PrepDataset's __init__ variables
        KerasBinaryClassification.__init__(self, X, y)

        self.input_shape = None
        self.include_lr = False
    
    # define the model
    def model_builder(self, hp):
        '''
        Args:
        hp - Keras tuner object
        '''
        self.n_layers = hp.Choice('n_layers', values = [6, 8, 10]) 
        self.UNITS = hp.Choice('units', values = [512, 1024, 2048])
        self.ACTIVATION = hp.Choice("activation", ["relu", "swish"])
        self.DROPOUT = hp.Choice("dropout_rate", [0.1, 0.2, 0.3])
    
        if self.include_lr == True:
            # Tune the learning rate for the optimizer
            # Choose an optimal value from 0.001, 0.0001, or 0.00001
            self.learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
       
        self.create_func_api_model()   # create model with Functional API method
        
        print()
        param_info = "{}Layers: {} | Units: {} | ACTIVATION: {} | DROPOUT: {} | LR: {}{}".format(
           color.bdgreen,
            self.n_layers,
            self.UNITS,
            self.ACTIVATION,
            self.DROPOUT,
            self.learning_rate,
            color.end
            )
        print(param_info)
        
        return self.model

print('keras_tuning class defined')

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="part1"></a>
<div style="font-family: Trebuchet MS;background-color:DarkRed;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
<h1 style='color:GhostWhite;'>Part 1. tf.Keras Models</h1>
</div>

<a id="functions"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Detect and Initialize TPU</b><br>
Kaggle's Tensor Processing Units (TPUs) are accelerators ideal for large workloads.  Using a TPU is a bit like using eight GPUs at once.<br><br>
For a 10 minute run with TPU active on Kaggle, it will take more than 24 hours without TPU. GPU is still fine, but for the Optuna run, better use TPU.  
</div>

In [ ]:
tf.keras.backend.clear_session()
tpu_strategy = InitTPUStrategy()

In [ ]:
# Setup plotting
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

print('Plotting defaults set up')

<a id="shared_functions"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Shared Functions</b><br>
    - used by different training processes further below
</div>

In [ ]:
def PredictUnseen(model_name = ''):
    ptd = TestPredict(model)
    ptd.PredictTestData(
                    X_test = dl.X_test,
                    y_test = dl.y_test,
                    label = f'Unseen Test Data : {model_name}'
                    )
    
print('PredictUnseen() defined')

In [ ]:
def UserInputTest():
    uit = TestPredict(model)

    '''
    Let's assume the user inputs client data for a company looking for a loan, then it is processed 
    by the program to output the user_input dictionary below, which is then used by the model for 
    prediction.

    These user inputs are actually extracted from the unseen X_test dataset.
    '''

    user_input = {  
                'Term':84,
                'NoEmp':39,
                'NewExist':1,
                'CreateJob':0 ,          
                'RetainedJob':0,         
                'FranchiseCode':1,       
                'UrbanRural':1,           
                'LowDoc':0,               
                'DisbursementGross':81039,                 
                'SBA_Appv':25000,          
                'Industry':33, 
                'Recession':0,
                'RealEstate':0,           
                'SBA_Portion':50,
                'State_hash':181805,
                'CityState_hash':203332
                }      
    # Test 1
    # prediction is expected to be 1 : approve the loan
    uit.UserInputTestPredict(testnum = 1, user_input = user_input)


    user_input = {  
                'Term':24, 
                'NoEmp':4,
                'NewExist':1,
                'CreateJob':0,          
                'RetainedJob':0,         
                'FranchiseCode':1,       
                'UrbanRural':1,           
                'LowDoc':0,               
                'DisbursementGross':50000,                 
                'SBA_Appv':25000,          
                'Industry':42, 
                'Recession':0,
                'RealEstate':0,           
                'SBA_Portion':50,
                'State_hash':321147,
                'CityState_hash':699085
                }
    # Test 2
    # prediction is expected to be 0 : do not approve the loan
    uit.UserInputTestPredict(testnum = 2, user_input = user_input)
    
print('UserInputTest() defined')

<a id="binary_classification"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>1.1 Binary Classification tf.keras model</h2>
</div>

Here we attempt to create a <b>relatively high capacity network</b>, both <b>wide</b> and <b>deep</b>.  

"A network's capacity refers to the size and complexity of the patterns it is able to learn. For neural networks, this will largely be determined by how many neurons it has and how they are connected together. If it appears that your network is underfitting the data, you should try increasing its capacity; and vice versa when overfitting.

Capacity can be increased either by making it wider (more units to existing layers) or by making it deeper (adding more layers). Wider networks have an easier time learning more linear relationships, while deeper networks prefer more nonlinear ones. Which is better just depends on the dataset."

Here we use more units (wide) and more layers (deep) to simulate a wide and deep network.

In [ ]:
def BinaryClassificationModel():
    tf.keras.backend.clear_session()
 
    # Process Data
    X = pd.read_feather(final_ds)
    #X = X.head(1000)      # uncomment for testing
    y = X.pop('MIS_Status')
    
    # Initiate the custom class, this includes initializing most parameters/variables
    dl = KerasBinaryClassification(X,y)
    dl.osample()                # oversample
    dl.split_data()             # split into three - train, valid, test
    dl.Scale_Data_Yeo_Johnson()

    dl.X, dl.y = None, None
    del X, y
    gc.collect();

    dl.input_shape = [dl.X_train.shape[1]]
    print()
    print(f'X_train shape: {dl.X_train.shape}')
    print(f"Input shape: {dl.input_shape}")  
    
    # Initialize the number of EPOCHS and early stopping value
    dl.EPOCHS = 200
    dl.EARLY_STOPPING_ROUNDS = dl.EPOCHS * 0.1   # 10%
    
    dl.n_layers = 6                  # how deep the model will be
    
    print(f'\n{color.bdblue}Parameters In Use{color.end}')
    param_info = "{}Layers: {} | Units: {} | ACTIVATION: {} | DROPOUT: {}{}".format(
            color.bdgreen,
            dl.n_layers,
            dl.UNITS,
            dl.ACTIVATION,
            dl.DROPOUT,
            color.end
            )
    print(param_info)
    
    #dl.create_seq_model()            # this creates a model using the Sequential method
    
    dl.create_func_api_model()       # this creates a model using the Functional API method
    
    print()
    history = dl.FitModel()          # fit the model
    
    # Save model in Tensorflow's "SavedModel" format
    save_locally = tf.saved_model.SaveOptions(experimental_io_device = '/job:localhost')
    dl.model.save(f'{workdir}ks_bc_model', options = save_locally)

    # Zip model directory.  Only doing this to be able to easily download from Kaggle working dir.
    ZipDir(f'{workdir}ks_bc_model')
    
    print()
    print(f'{color.bdblue}Model has been saved to {workdir}ks_bc_model{color.end}')
    
    # Plot model structures
    print()
    print(f'{color.bdblue}Model structure{color.end}')
    display(tf.keras.utils.plot_model(dl.model,\
                              f'{workdir}ks_bc_model.png',show_shapes=True))
    
    print()
    print(f'{color.bdblue}Model Summaries:{color.end}')
    display(dl.model.summary())

    # Plot Learning Rate history.  Comment out if not using CyclicLR() callback
    h = dl.clr_triangular.history
    lr = h['lr']
    #print(f'lr History: {lr}')
    plt.title('Learning Rate History')
    plt.plot(dl.clr_triangular.history['iterations'], dl.clr_triangular.history['lr'])
  
    # Save and Plot training history
    hdf = pd.DataFrame(history.history)    
    hdf = dl.SavePlotHistory(hdf, 'hdf_ks_bc_model.csv.feather')
    
    # initialize return value
    bc_dict = { 'ks_bc_model':dl.model,
                'ks_history':hdf,
                'dl_class':dl}

    # return value
    return bc_dict

print('BinaryClassificationModel() defined')

In [ ]:
%%time
bc_dict = BinaryClassificationModel()

In [ ]:
# Uncomment to load model in Tensorflow's "SavedModel" format
#load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
#bc_model = tf.keras.models.load_model(f'{workdir}ks_bc_model', options=load_locally) 
#model_weights = bc_model.get_weights()

In [ ]:
model = bc_dict['ks_bc_model']
dl = bc_dict['dl_class']

<a id="test_unseen"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Test Binary Classification Model with Unseen Dataset</b>
</div>

In [ ]:
PredictUnseen(model_name = 'Keras Binary Classification')

<a id="uit"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Test Binary Classification Model with Single User Inputs</b>
</div>

In [ ]:
UserInputTest()

In [ ]:
# uncomment if needed
#del bc_dict, model, dl
#gc.collect()

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="binary_classification_wide_and_deep"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>1.2 Binary Classification Wide and Deep tf.keras model</h2>
</div>

<div class="alert alert-block alert-info">
A <b>Wide and Deep</b> network trains a linear layer side-by-side with a deep stack of dense layers. Wide and Deep networks are often effective on tabular datasets.

Here we use the <b>keras.experimental.WideDeepModel</b> algorithm.  However, in comparison to the previous 1.1 Binary Classification tf.keras model, which is "wide" in having an increased capacity with more units to existing layers, and "deep" with more layers, the resulting scores are more or less similar.
</div>

In [ ]:
'''
The parameter best_trial and tuner is set to None by default.
tuner:
   op : Optuna
   kt : Keras Tuner
When Optuna or Keras Tuner is run, they will output a best_trial dictionary. This dictionary 
will then be passed to WideAndDeepModel() to test the Optuna best_trial hyperparameter set.
'''
def WideAndDeepModel(tuner_type = None, best_trial = None):
    tf.keras.backend.clear_session()
    X = pd.read_feather(final_ds)
    #X = X.head(1000)      # uncomment for testing
    y = X.pop('MIS_Status')

    # initiate the custom class, this includes initializing most parameters/variables
    dl = KerasBinaryClassification(X,y)
    dl.osample()
    dl.split_data()
    dl.Scale_Data_Yeo_Johnson()
    
    dl.X, dl.y = None, None
    del X, y
    gc.collect()

    dl.input_shape = [dl.X_train.shape[1]]
    print()
    print(f'X_train shape: {dl.X_train.shape}')
    print("Input shape: {}".format(dl.input_shape))  
 
    '''Initialize the number of EPOCHS and early stopping value'''
    dl.EPOCHS = 200
    dl.EARLY_STOPPING_ROUNDS = dl.EPOCHS * 0.1   # 10%
      
    dl.n_layers = 6

    # if WideAndDeepModel() is called when runing Optuna or Keras Tuner
    if tuner_type != None:
        print('{}{} best_trial parameters have been set for a WideAndDeep model{}'.format(
                    color.bdblue,
                    tuner_type,
                    color.end
                    )
             )
        dl.n_layers = best_trial['n_layers']
        dl.UNITS = best_trial['units']
        dl.ACTIVATION = best_trial['activation']
        dl.DROPOUT = best_trial['dropout_rate']
        if 'learning_rate' in best_trial:
            dl.kt_run = True   # if True, no learning_rate schedule callback in fit()
            dl.learning_rate = best_trial['learning_rate']
            lr = dl.learning_rate
        else:
            lr = 'Not Included'
            
        param_info =\
            "{}Layers: {} | Units: {} | ACTIVATION: {} | DROPOUT: {} | LR: {}{}".format(
                    color.bdgreen,
                    dl.n_layers,
                    dl.UNITS,
                    dl.ACTIVATION,
                    dl.DROPOUT,
                    lr,
                    color.end
                    )
    else:
        print(f'{color.bdblue}Parameters In Use{color.end}')
        param_info = "{}Layers: {} | Units: {} | ACTIVATION: {} | DROPOUT: {}{}".format(
            color.bdgreen,
            dl.n_layers,
            dl.UNITS,
            dl.ACTIVATION,
            dl.DROPOUT,
            color.end
            )
    
    print(param_info)
    
    dl.create_wide_and_deep_model()
    
    print()
    
    history = dl.FitModel()   # fit the model
        
    # Save in Tensorflow's "SavedModel" format
    save_locally = tf.saved_model.SaveOptions(experimental_io_device = '/job:localhost')

    dl.wide.save(f'{workdir}ks_wide_model', options = save_locally)
    dl.deep.save(f'{workdir}ks_deep_model', options = save_locally)
    dl.wide_and_deep.save(f'{workdir}ks_wide_and_deep_model', options = save_locally)

    # Zip model directory.  
    #Only doing this to be able to easily download from Kaggle working dir.
    ZipDir(f'{workdir}ks_wide_model')
    ZipDir(f'{workdir}ks_deep_model')
    ZipDir(f'{workdir}ks_wide_and_deep_model')
    
    print()
    print(f'{color.bdblue}Models have been saved to directory {workdir}{color.end}')
  
    # Plot model structures
    print()
    print(f'{color.bdblue}Model structure{color.end}')
    print(f'{color.bdgreen}Wide model{color.end}')
    display(tf.keras.utils.plot_model(dl.wide,\
                              f'{workdir}ks_wide_model.png',show_shapes=True))
    print(f'{color.bdgreen}Deep model{color.end}')
    display(tf.keras.utils.plot_model(dl.deep,\
                              f'{workdir}ks_deep_model.png',show_shapes=True))
    print(f'{color.bdgreen}Wide and Deep model{color.end}')
    display(tf.keras.utils.plot_model(dl.wide_and_deep,\
                              f'{workdir}ks_wide_and_deep_model.png',show_shapes=True))
    
    print()
    print(f'{color.bdblue}Model Summaries:{color.end}')
    display(dl.wide_and_deep.summary())
    print()
    display(dl.wide.summary())
    print()
    display(dl.deep.summary())
    print()

    # Save and Plot history
    hdf = pd.DataFrame(history.history)
    if tuner_type == None:
        hdf_savename = 'hdf_ks_wd_model.csv.feather'
        #print(f'{color.bdgreen}Wide And Deep History saved to {hdf_savename}{color.end}')
    else:
        if tuner_type == 'op':
            hdf_savename = 'hdf_optuna_ks_wd_model.csv.feather'
        else:
            hdf_savename = 'hdf_keras_tuner_ks_wd_model.csv.feather'
        #print(f'{color.bdgreen}Wide And Deep Optuna History saved to {hdf_savename}{color.end}')
    hdf = dl.SavePlotHistory(hdf, hdf_savename)

    # initialize return value
    wd_dict = { 'ks_wide_and_deep_model':dl.wide_and_deep,
                'ks_deep_model':dl.deep,
                'ks_wide_model':dl.wide,
                'ks_history':hdf,
                'dl_class':dl}
    return wd_dict

print('WideAndDeepModel defined')

In [ ]:
%%time
wd_dict = WideAndDeepModel()

In [ ]:
model = wd_dict['ks_wide_and_deep_model']
dl = wd_dict['dl_class']

In [ ]:
display(model.metrics_names)
display(model.loss)
display(model.name)

<a id="test_unseen_wd"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Test keras.experimental.WideDeepModel with Unseen Dataset</b>
</div>

In [ ]:
PredictUnseen(model_name = 'keras.experimental.WideDeepModel')

<a id="uit_wd"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Test keras.experimental.WideDeepModel with Single User Inputs</b>
</div>

In [ ]:
# this function should have been defined earlier before running

UserInputTest()

In [ ]:
# uncomment if needed
#del wd_dict, model, dl
#gc.collect()

<a id="compare_bc_wd"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>1.3 Score Comparison : Binary Classification vs WideAndDeep Models</h2>
</div>

In [ ]:
def CompareModels():
    hdf1 = pd.read_feather(f'{workdir}hdf_ks_bc_model.csv.feather')
    hdf2 = pd.read_feather(f'{workdir}hdf_ks_wd_model.csv.feather')
    
    crc = CompareResults()  # instantiate custom class

    crc.BuildCompare(hdf1, hdf2,
                 'Binary Classification',     # column 1 name
                 'WideAndDeep',               # column 2 name
                 'Standard Binary Classification vs WideAndDeep Scores'      # title
                )
    
CompareModels()

<div class="alert alert-block alert-info">
The scores between using the simulated wide and deep Binary Classification model versus the keras.experimental.WideDeepModel model are <b>very similar</b>.
</div>

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="part2"></a>
<div style="font-family: Trebuchet MS;background-color:DarkRed;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
<h1 id="part2" style='color:GhostWhite;'>Part 2. Optuna Hyperparameter Tuning
</div>

<a id="optuna_study"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.1 Optuna Study</h2>
</div>

In [ ]:
#from optuna.integration import KerasPruningCallback
# For running Optuna tuning on full dataset.
def OptunaStudyKeras():
    tf.keras.backend.clear_session()
    X = pd.read_feather(final_ds)
    #X = X.head(1000)      # uncomment for testing
    y = X.pop('MIS_Status')

    features_num = list(X)
    print(features_num)
    
    # initiate the custom class
    dl = optuna_tuning(X,y)
    dl.osample()
    dl.split_data()
    dl.Scale_Data_Yeo_Johnson()
 
    dl.X, dl.y = None, None
    del X, y
    gc.collect()

    dl.input_shape = [dl.X_train.shape[1]]
    print()
    print("Input shape: {}".format(dl.input_shape))  

    if os.path.exists(f'{workdir}ks_optuna_study_log.txt'):
        os.remove(f'{workdir}ks_optuna_study_log.txt')
    
    #*********************************************************
    # Initialize number of trials and epochs
    dl.nn_trials = None         
    dl.nn_timeout = 60 * 240    # 3 hours  
    dl.EPOCHS = 200
    dl.EARLY_STOPPING_ROUNDS = dl.EPOCHS * 0.1   # 10%
    #*********************************************************
    
    # STUDY
    
    # Turn off optuna log notes, to use own logging notes
    #optuna.logging.set_verbosity(optuna.logging.WARN)
    optuna.logging.set_verbosity(optuna.logging.INFO)

    print(f"{color.bold}Please wait, finding best trial ...{color.end}")         
    dl.gt = dt.datetime.now()   # to show total runtime later
    
    study = optuna.create_study(direction="maximize")
    try:
        # callbacks [self.save_study] is to save study in case memory fails
        study.optimize(dl.objective_keras,
                        n_trials = dl.nn_trials,
                        timeout = dl.nn_timeout,
                        callbacks = [dl.logging_callback, dl.save_study],
                        gc_after_trial = True,
                        catch = (RuntimeWarning,ArithmeticError,))
    except MemoryError as e:
        print(f'{color.bdblue}{e} : Memory was getting low, Trial ended early{color.end}')
        
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    print()
    print(f'Number of finished trials: {len(study.trials)}')
    print(f'Number of pruned trials: {len(pruned_trials)}')
    print(f'Number of completed trials: {color.bdgreen}{len(complete_trials)}{color.end}')
    print(f'{color.bdblue}Best trial: {study.best_trial.params}{color.end}')
    
    joblib.dump(study, f"{workdir}ks_optuna_study.pkl")   # save study

    print()
    return {'study':study}

print('OptunaStudyKeras() has been defined')

In [ ]:
%%time
if optuna_flag == 1:
    study_results = OptunaStudyKeras()

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="optuna_metrics"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.2 Optuna Study Metrics</h2>
</div>

In [ ]:
# View the trials log; Very useful to know which parameters one can avoid
with open(f"{workdir}ks_optuna_study_log.txt", "r+") as os_log:
    print(os_log.read())

In [ ]:
# uncomment below if you want to load saved study to check, if desired
jl = joblib.load(f"{workdir}ks_optuna_study.pkl")
jl = jl.best_trial.params
display(jl)

In [ ]:
# one can get the best trial by loading the saved study
study = joblib.load(f"{workdir}ks_optuna_study.pkl")
best_trial = study.best_trial.params
display(best_trial)

In [ ]:
# here, we just use the study returned by OptunaStudyKeras() previously
best_trial = study_results['study'].best_trial.params
best_trial

In [ ]:
# Trial results dataframe sorted from best value (RMSE) ascending
def ViewResultsAsDf():
    stdf = study_results['study'].trials_dataframe()
    stdf = stdf.sort_values('value',ascending=True)

    return stdf.head(2)    # return here is only used for printing output

ViewResultsAsDf()

In [ ]:
#Visualize parameter importance
optuna.visualization.plot_param_importances(study)

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="try_best_hp"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.3 Test Optuna Best Trial Hyperparameters</h2>
</div>

In [ ]:
%%time

wd_dict = WideAndDeepModel(tuner_type = 'op', best_trial = best_trial)

<a id="optuna_comparison"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.4 Optuna Tuning Comparison</h2>
</div>

<div class="alert alert-block alert-info">
    <b>Compare metrics before and after Optuna tuning.</b><br><br>
    Comparison is made between the WideAndDeep model results before and after Optuna tuning.
  </div>

In [ ]:
def CompareOptuna():
    hdf1 = pd.read_feather(f'{workdir}hdf_ks_wd_model.csv.feather')
    hdf2 = pd.read_feather(f'{workdir}hdf_optuna_ks_wd_model.csv.feather')
    
    crc = CompareResults()  # instantiate custom class

    crc.BuildCompare(hdf1, hdf2,
                 'Before Optuna',     # column 1 name
                 'After Optuna',      # column 2 name
                 'Binary Accuracy Improvement Using Optuna Suggested Parameters:'   # title
                )
    
CompareOptuna()

<div class="alert alert-block alert-warning">
    I used <b>4 hours of Optuna training time</b>, with TPU as accelerator, and could have had a better score with a longer time which would mean more trial runs.  We could also repeat the run.  Anyway, this is just to demonstrate Optuna Tuning for Keras.<br><br>
Also, it seems that I already chose a good hyperparameter set to start with, before Optuna.
</div>

In [ ]:
model = wd_dict['ks_wide_and_deep_model']
dl = wd_dict['dl_class']

<a id="optuna_test_unseen_wd"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Test Optuna-Tuned keras.experimental.WideDeepModel with Unseen Dataset</b>
</div>

In [ ]:
PredictUnseen(model_name = 'Optuna-tuned keras.experimental.WideDeepModel')

<a id="optuna_uit"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Test Optuna-Tuned keras.experimental.WideDeepModel with Single User Inputs</b>
</div>

In [ ]:
UserInputTest()

In [ ]:
# uncomment if needed
#del best_trial, study_results, wd_dict, model, dl
#gc.collect()

In [ ]:
if alert_flag == 1:
    if kaggle_flag == 0:   # not Kaggle
        engine.say("Optuna run completed.")
        engine.runAndWait()
    else:
        display(Audio(url=audio_path, autoplay=True))

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>

<a id="part3"></a>
<div style="font-family: Trebuchet MS;background-color:DarkRed;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
<h1 id="part2" style='color:GhostWhite;'>Part 3. Keras Tuner Hyperparameter Tuning
    </div>

<div class="alert alert-block alert-warning">
I favor Optuna more than Keras Tuner, especially due to Optuna's speed.  At any rate, below is the code demonstration on how to use Keras Tuner, for reference.  It is also significantly slower than Optuna, even with TPU.<br><br>
As at May 2022, Kaggle Tuner as is will fail in Kaggle, as it will be unable to save epoch checkpoints to Kaggle's working folder, it can only save to a Google Cloud Storage (gcs) location.  As a temporary solution, we mock/patch the Kaggle Tuner function that is responsible, and this solves the issue.
</div>

<a id="kt_study"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>3.1 Keras Tuner Search</h2>
    Here we use the BayesianOptimization method, which is slower than Optuna.  We could also have used the faster Hyperband method (more like RandomSearch).   
</div>

<b>HyperBand</b><br>
"This technique tries to remove one of the problems in random search of hyperparameters. Problem is as follows: Random search may pick some values which are very obviously bad and will do full training and evaluation on it, which is wasteful. Hyperband provides one way to solve this problem. 

Hyperband Solution: Randomly sample all the combinations of hyperparameter and now instead of running full training and evaluation on it, train the model for few epochs (less than max_epochs) with these combinations and select the best candidates based on the results on these few epochs. It does this iteratively and finally runs full training and evaluation on the final chosen candidates. The number of iterations done depends on parameter ‘hyperband_iterations’ and number of epochs in each iteration are less than ‘max_epochs’."<br><br>
<b>Bayesian Optimization</b><br>
"This techniques addresses a common problem in RandomSearch and Hyperband. Problem: All the hyperparameter combinations are chosen randomly. Choosing hyperparameters randomly helps to explore the hyperparameter space but does not guarantee absolute optimal hyperparameters. 

Bayesian Ozptimization Solution: Instead of all combinations being random, it chooses first few randomly, then based on the performance on these hyperparameters it chooses the next best possible hyperparameters. <b>Hence it takes into account the history of the hyperparameters which were tried.</b> The iterations of choosing next set of hyperparameters based on history and evaluating performance continues till the tuner reaches optimal hyperparameters or exhausts maximum number of allowed trails."<br>
- <a href="https://medium.com/swlh/hyperparameter-tuning-in-keras-tensorflow-2-with-keras-tuner-randomsearch-hyperband-3e212647778f"><i>from Hyperparameter Tuning in Keras</i></a>

In [ ]:
def KerasTuner(tuner_to_use = 'bo'):
    tf.keras.backend.clear_session()
    X = pd.read_feather(final_ds)
    X = X.head(1000)      # uncomment for testing
    y = X.pop('MIS_Status')
    
    # initiate the custom class
    dl = keras_tuning(X,y)
    dl.osample()
    dl.split_data()
    dl.Scale_Data_Yeo_Johnson()
 
    dl.X, dl.y = None, None
    del X, y
    gc.collect()

    dl.input_shape = [dl.X_train.shape[1]]
    print()
    print("Input shape: {}".format(dl.input_shape))  

    #***********************************************************
    # Initialize number of trials and epochs
    dl.EPOCHS = 2
    dl.EARLY_STOPPING_ROUNDS = dl.EPOCHS * 0.1   # 10%
    
    '''include learning_rate in tuning.  Personally,
    I prefer to use an lr_scheduler, but we include it here
    for demonstration purpose'''
    dl.include_lr = True
                     
    if tuner_to_use == 'bo':
        nn_trials = 2       # only for BayesianOptimization
        
    #************************************************************

    # TUNER SEARCH

    print(f"{color.bold}Please wait, finding best trial ...{color.end}")         
    
    if tuner_to_use == 'bo':
        # Instantiate the search
        # Here, we use BayesianOptimization.  The hypermodel function builds the model,
        # and calls the compile method as well as initialize the early_stopping callback
        tuner = kt.BayesianOptimization(
            dl.model_builder,                  # the hypermodel, found in custom class keras_tuning 
            objective = 'val_binary_accuracy', # objective to optimize
            max_trials = nn_trials,            # number of trials
            distribution_strategy = tpu_strategy,
            # this project name (a dir) is created in 'directory', and is used to 
            #     resume tuning if the search is run again
            directory = f'{workdir}',   
            project_name = 'kbayesian',
            # if True, overwrite above directory if search is run again - i.e. don't resume
            overwrite = True
        )     
    else:    
        # Here, we use HyperBand
        tuner = kt.Hyperband(
            dl.model_builder,
            objective = 'val_binary_accuracy',
            max_epochs = dl.EPOCHS,
            factor = 30000000000,
            # One iteration will run approximately
            #     max_epochs * (math.log(max_epochs, factor) ** 2) 
            #     cumulative epochs across all trials
            hyperband_iterations = 1,    
            distribution_strategy = tpu_strategy,
            directory = f'{workdir}',         
            project_name = 'khyperband',
            overwrite = True
         ) 

    # hypertuning settings
    print()
    display(tuner.search_space_summary())
    
    #*****************************************************************************************
    '''
    When we run the keras-tuner search for hyperparameters with TPU accelerator in Kaggle, we 
    will get the error: 'UnimplementedError: File system scheme '[local]' not implemented', since
    keras-tuner does not have the option to save checkpoints to a locally - i.e. the Kaggle working 
    directory.
    
    To fix this, we can use a gcs location under directory parameter, and that's it.
    
    However, if we still want to save to the Kaggle working directory, then we mock / patch the 
    keras-tuner function that saves checkpoints, using our own function temporarily to allow 
    saving locally.  
    
    First of all, Keras Tuner version must be 1.0.3 and above.
    
    Then, here we "replace" the function on_epoch_end() with new_on_epoch_end().
    This function is under keras_tuner.engine.tuner_utils.SaveBestEpoch class.
    
    We use "with patch()", which will temporarily replace the function with our code, and 
    revert it back automatically after use.
    '''
    #*****************************************************************************************
    def new_on_epoch_end(self, epoch, logs=None):
        if not self.objective.has_value(logs):
            # Save on every epoch if metric value is not in the logs. Either no
            # objective is specified, or objective is computed and returned
            # after `fit()`.
            
            '''we comment out the original code'''
            #self.model.save_weights(self.filepath)
             
            '''we put in our code'''
            # Save model in Tensorflow's "SavedModel" format
            save_locally = tf.saved_model.SaveOptions(experimental_io_device = '/job:localhost')
            self.model.save_weights(self.filepath, options = save_locally)

            return
        current_value = self.objective.get_value(logs)
        if self.objective.better_than(current_value, self.best_value):
            self.best_value = current_value
            
            '''we comment out the original code'''
            #self.model.save_weights(self.filepath)
            
            '''we put in our code'''
            # Save model in Tensorflow's "SavedModel" format
            save_locally = tf.saved_model.SaveOptions(experimental_io_device = '/job:localhost')
            self.model.save_weights(self.filepath, options = save_locally)    
    
    '''
    with patch() will now make tuner.search() use our patch, and automatically revert back
    to original after use
    '''
    with patch('keras_tuner.engine.tuner_utils.SaveBestEpoch.on_epoch_end', new_on_epoch_end):
        # Perform hypertuning.  The parameters are exactly like those in the fit() method.
        tuner.search(
            dl.X_train,
            dl.y_train,
            epochs=dl.EPOCHS,
            validation_data = (dl.X_valid, dl.y_valid), 
            callbacks=[dl.early_stopping]   # found at KerasBinaryClassification.PrepKSModel()
            )
        
    return {
            'tuner':tuner,
            'dl':dl,
           }

print('KerasTuner() defined')

In [ ]:
%%time
if kt_flag == 1:
    '''
    bo = BayesianOptimization
    hb = Hyperband
    '''
    kt_retval = KerasTuner(tuner_to_use = 'bo')

<a id="kt_metrics"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>3.2 Keras Tuner Metrics</h2>
</div>

In [ ]:
tuner.results_summary(num_trials=10)

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
best_trial = best_hp.values
best_trial

<a id="kt_try_best_hp"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>3.3 Test Keras Tuner's Best Hyperparameters</h2>
</div>

In [ ]:
%%time

wd_dict = WideAndDeepModel(tuner_type = 'kt', best_trial = best_trial)

<a id="kt_comparison"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>3.4 Keras Tuner Tuning Comparison</h2>
</div>

<div class="alert alert-block alert-info">
    <b>Compare metrics before and after Keras Tuner tuning.</b><br><br>
    Comparison is made between the WideAndDeep model results before and after Keras Tuner tuning.
  </div>

In [ ]:
def CompareKerasTuner():
    hdf1 = pd.read_feather(f'{workdir}hdf_ks_wd_model.csv.feather')
    hdf2 = pd.read_feather(f'{workdir}hdf_keras_tuner_ks_wd_model.csv.feather')
    
    crc = CompareResults()  # instantiate custom class

    crc.BuildCompare(hdf1, hdf2,
                 'Before KT',     # column 1 name
                 'After KT',      # column 2 name
                 'Binary Accuracy Improvement Using KT Suggested Parameters:'   # title
                )

CompareKerasTuner()

In [ ]:
model = wd_dict['ks_wide_and_deep_model']
dl = wd_dict['dl_class']

<a id="kt_test_unseen_wd"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Test KT-tuned keras.experimental.WideDeepModel with Unseen Dataset</b>
</div>

In [ ]:
PredictUnseen(model_name = 'Keras Tuner with keras.experimental.WideDeepModel')

In [ ]:
UserInputTest()

<i><a style="color:DarkSlateGrey" href="#toc">Back to Table Of Contents</a></i>